In [1]:
!pip install -q yfinance

     |████████████████████████████████| 6.3MB 7.6MB/s 


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

In [4]:
def fix_col_names(df):
  return ['IBOV' if col =='^BVSP' else col.rstrip('.SA') for col in df.columns]

In [8]:
prices = yf.download('^BVSP MGLU3.SA TAEE11.SA', period='5y')['Adj Close']
prices.columns = fix_col_names(prices)
prices

[*********************100%***********************]  3 of 3 completed


,MGLU3,TAEE11,IBOV
Date,,,
2016-05-17,0.127636,11.476492,50839.000000
2016-05-18,0.129950,11.697655,50562.000000
2016-05-19,0.128681,11.356947,50133.000000
2016-05-20,0.130584,11.619948,49723.000000
2016-05-23,0.125920,11.464538,49330.000000
...,...,...,...
2021-05-11,19.320000,38.240002,122964.000000
2021-05-12,18.590000,37.380001,119710.000000
2021-05-13,19.129999,38.099998,120706.000000


In [14]:
retorno = np.log(prices/prices.shift(1)).dropna()

In [15]:
retorno

,MGLU3,TAEE11,IBOV
Date,,,
2016-05-18,0.017962,0.019088,-0.005463
2016-05-19,-0.009809,-0.029559,-0.008521
2016-05-20,0.014678,0.022894,-0.008212
2016-05-23,-0.036369,-0.013465,-0.007935
2016-05-24,0.023723,-0.002088,0.000304
...,...,...,...
2021-05-11,0.006230,0.002094,0.008617
2021-05-12,-0.038517,-0.022746,-0.026819
2021-05-13,0.028634,0.019078,0.008286


In [16]:
# Gráfico de Retornos
fig = go.Figure()

fig.layout.title='Retorno Diário' 

for ativo in retorno.columns:
  fig.add_scatter(x=retorno.index,y=retorno[ativo], name=ativo)
 
# fig.show()

In [17]:
# Histograma IBOV

fig = px.histogram(x=retorno['IBOV'], 
             title='Histograma do Retorno Diário do IBOV')

fig.layout.xaxis.title = 'Retorno Log'
fig.layout.yaxis.title = 'Ocorrências'

fig.update_layout(bargap=0.1)

In [18]:
fig = go.Figure()

fig.layout.title='Histograma de Retornos Diários' 

for ativo in retorno.columns:
  fig.add_histogram(x=retorno[ativo], name=ativo)
 
fig.layout.xaxis.title = 'Retorno Log'
fig.layout.yaxis.title = 'Ocorrências'

fig.show()

In [19]:
# Qual o retorno médio diário?
retorno_medio = retorno.mean()
retorno_medio

MGLU3     0.004197
TAEE11    0.000992
IBOV      0.000703
dtype: float64

In [20]:
# Qual a variância em relação a média?
variancia = retorno.var()
variancia

MGLU3     0.001334
TAEE11    0.000239
IBOV      0.000301
dtype: float64

In [21]:
# Qual o desvio-padrão em relação à média?
desvio_padrao = retorno.std()
desvio_padrao

MGLU3     0.036525
TAEE11    0.015447
IBOV      0.017360
dtype: float64

In [22]:
# Matriz de Covariância
retorno.cov()

,MGLU3,TAEE11,IBOV
MGLU3,0.001334,0.000154,0.000354
TAEE11,0.000154,0.000239,0.000126
IBOV,0.000354,0.000126,0.000301


In [23]:
# Matriz de Correlação
retorno.corr()

,MGLU3,TAEE11,IBOV
MGLU3,1.000000,0.272722,0.557581
TAEE11,0.272722,1.000000,0.469549
IBOV,0.557581,0.469549,1.000000


### Anualizando 

No mercado financeiro é muito comum trabalhar com essas métricas por ano, considerando 252 dias úteis no ano

#### $retorno_{ano} = retorno_{dia} * 252$

In [59]:
print(f'Retorno médio\n{retorno_medio}')



Retorno médio
MGLU3     0.004197
TAEE11    0.000992
IBOV      0.000703
dtype: float64


#### $var_{ano} = var_{dia} * 252$

In [25]:
variancia * 252

MGLU3     0.336179
TAEE11    0.060127
IBOV      0.075946
dtype: float64

#### $cov_{ano} = cov_{dia} * 252$

In [ ]:
retorno.cov()*252

#### $\sigma_{ano} = \sigma_{dia} * \sqrt{252}$

In [60]:
desvio_padrao * np.sqrt(252)

MGLU3     0.579810
TAEE11    0.245208
IBOV      0.275582
dtype: float64